In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.callbacks import Callback

In [ ]:
data = pd.read_excel('rice_yield_final_sheet.xlsx')
data = data[(data['Yield'] <= 4.5) & (data['Yield'] >= 1.5)]

In [ ]:
#Visualize the data
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 10))

sample=data[[ 'Maxtemp','Mintemp','Avgtemp']]
axes[0, 0].boxplot(sample.values, vert=True)
axes[0, 0].set_xticklabels(['Maximum\nTemperature', 'Minimum\nTemperature', 'Average\nTemperature'], fontsize=14)
axes[0, 0].set_title("Temperatures", fontsize=14)

sample=data[[ 'Humid', 'Soil_Moisture']]
axes[0, 1].boxplot(sample.values, vert=True, labels=sample.columns)
axes[0, 1].set_xticklabels(['Humidity', 'Soil Moisture'], fontsize=14)
axes[0, 1].set_title("Humidity", fontsize=14)

sample=data[[ 'Rainfall']]
axes[1, 0].boxplot(sample.values, vert=True, labels=sample.columns)
axes[1, 0].set_xticklabels(['Rainfall'], fontsize=14)
axes[1, 0].set_title('Rainfall', fontsize=14)

sample=data[[ 'Evapotranspiration']]
axes[1, 1].boxplot(sample.values, vert=True, labels=sample.columns)
axes[1, 1].set_xticklabels(['Evapotranspiration'], fontsize=14)
axes[1, 1].set_title('Evapotranspiration', fontsize=14)

plt.tight_layout()
plt.savefig('Rice_Visualized_Data.png')
plt.show()

In [ ]:
#Removing Outliers
sample = data['Evapotranspiration'].values

Q1 = np.percentile(sample, 25)
Q3 = np.percentile(sample, 75)

IQR = Q3 - Q1

lower_whisker = Q1 - 1.5 * IQR
upper_whisker = Q3 + 1.5 * IQR

data['Evapotranspiration'] = data['Evapotranspiration'].apply(lambda x: lower_whisker if x < lower_whisker else x)

sample = data['Soil_Moisture'].values

Q1 = np.percentile(sample, 25)
Q3 = np.percentile(sample, 75)

IQR = Q3 - Q1

lower_whisker = Q1 - 1.5 * IQR
upper_whisker = Q3 + 1.5 * IQR

data['Soil_Moisture'] = data['Soil_Moisture'].apply(lambda x: lower_whisker if x < lower_whisker else x)

In [ ]:
X = data[['Rainfall', 'Evapotranspiration', 'Soil_Moisture', 'Maxtemp','Mintemp','Avgtemp','Humid']]
y = data['Yield']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

result_df = pd.DataFrame(columns=['Model', 'RMSE', 'MSE', 'R2'])

#Linear Model
model = LinearRegression()

model.fit(X_train, y_train)
y_pred_lr = model.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_lr))
test_r2 = r2_score(y_test, y_pred_lr)
mse = mean_squared_error(y_test, y_pred_lr)
mae = mean_absolute_error(y_test, y_pred_lr)

new_data = {'Model': 'Linear Regression', 'MAE': mae,'RMSE': test_rmse, 'MSE': mse, 'R2': test_r2}
result_df = result_df.append(new_data, ignore_index=True)

# KNN
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_knn))
mse = mean_squared_error(y_test, y_pred_knn)
r2 = r2_score(y_test, y_pred_knn)
mae = mean_absolute_error(y_test, y_pred_knn)

new_data = {'Model': 'K-Nearest Neighbor', 'MAE': mae,'RMSE': test_rmse, 'MSE': mse, 'R2': r2}
result_df = result_df.append(new_data, ignore_index=True)


#Decision Tree
dtr = DecisionTreeRegressor(random_state=42)
dtr.fit(X_train, y_train)
y_pred_dtr = dtr.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_dtr))
mse = mean_squared_error(y_test, y_pred_dtr)
r2 = r2_score(y_test, y_pred_dtr)
mae = mean_absolute_error(y_test, y_pred_dtr)

new_data = {'Model': 'Decision Tree', 'MAE': mae,'RMSE': test_rmse, 'MSE': mse, 'R2': r2}
result_df = result_df.append(new_data, ignore_index=True)

#Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=20)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mse = mean_squared_error(y_test, y_pred_rf)
r2 = r2_score(y_test, y_pred_rf)
mae = mean_absolute_error(y_test, y_pred_rf)

new_data = {'Model': 'Random Forest', 'MAE': mae,'RMSE': test_rmse, 'MSE': mse, 'R2': r2}
result_df = result_df.append(new_data, ignore_index=True)

#eXtreme Gradient Boosting
xgb = XGBRegressor(n_estimators= 100, max_depth= 3, learning_rate= 0.15, subsample= 0.7, colsample_bytree= 0.9,
    objective= 'reg:squarederror', random_state= 42)
xgb.fit(X_train, y_train)
y_pred_xbg = xgb.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_xbg))
mse = mean_squared_error(y_test, y_pred_xbg)
r2 = r2_score(y_test, y_pred_xbg)
mae = mean_absolute_error(y_test, y_pred_xbg)

new_data = {'Model': 'Extreme Gradient Boosting', 'MAE': mae,'RMSE': test_rmse, 'MSE': mse, 'R2': r2}
result_df = result_df.append(new_data, ignore_index=True)


#Gradient Boosting Regressor
gbdt = GradientBoostingRegressor(n_estimators=300, learning_rate=0.01, subsample= 0.9, 
                                 max_depth=5, min_samples_leaf= 4, random_state=42)
gbdt.fit(X_train, y_train)
y_pred_gdbt = gbdt.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_gdbt))
mse = mean_squared_error(y_test, y_pred_gdbt)
r2 = r2_score(y_test, y_pred_gdbt)
mae = mean_absolute_error(y_test, y_pred_gdbt)

new_data = {'Model': ' Gradient Boosting Regressor', 'MAE': mae,'RMSE': test_rmse, 'MSE': mse, 'R2': r2}
result_df = result_df.append(new_data, ignore_index=True)

#Support Vector Regressor
svr_rbf = SVR(kernel='rbf', C=10, gamma='scale', epsilon=.1, degree=2)
svr_rbf.fit(X_train, y_train)
y_pred_svr = svr_rbf.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_svr))
mse = mean_squared_error(y_test, y_pred_svr)
r2 = r2_score(y_test, y_pred_svr)
mae = mean_absolute_error(y_test, y_pred_svr)

new_data = {'Model': 'Support Vector Regression', 'MAE': mae,'RMSE': test_rmse, 'MSE': mse, 'R2': r2}
result_df = result_df.append(new_data, ignore_index=True)

df_sorted = result_df.sort_values(by='R2')
df_sorted

In [ ]:
#ANN
class R2Callback(Callback):
    def __init__(self, training_data, validation_data):
        self.x_train, self.y_train = training_data
        self.x_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs=None):
        y_pred_train = self.model.predict(self.x_train)
        y_pred_val = self.model.predict(self.x_val)

        r2_val = r2_score(self.y_val, y_pred_val)
        rmse = np.sqrt(mean_squared_error(self.y_val, y_pred_val))
        mse = mean_squared_error(self.y_val, y_pred_val)

        print(f"Epoch {epoch + 1}: R²: {r2_val:.6f}, RMSE: {rmse:.6f}, MSE: {mse:.6f},")


r2_callback = R2Callback(training_data=(X_train, y_train), validation_data=(X_test, y_test))

# Build the model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=1, activation='linear'))


model.compile(optimizer='adam', loss='mse')

history = model.fit(X_train, y_train, epochs=200, batch_size=64, validation_split=0.1, verbose=1, validation_data=(X_test, y_test), callbacks=[r2_callback])

y_pred = model.predict(X_test)

test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Test RMSE: {test_rmse}")
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

In [ ]:
#LSTM
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

class R2Callback(Callback):
    def __init__(self, validation_data):
        super(R2Callback, self).__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        X_val, y_val = self.validation_data
        y_pred = self.model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        mse = mean_squared_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        print(f"\nEpoch {epoch+1}: R² = {r2:.6f}; mse = {mse:.6f}; rmse = {rmse:.6f}")

# Step 3: Define and compile the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

r2_callback = R2Callback(validation_data=(X_test, y_test))
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), batch_size=1, callbacks=[r2_callback])
loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {loss}')
y_pred = model.predict(X_test)

test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test RMSE: {test_rmse}")
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

In [ ]:
result_df['RMSE'] = result_df['RMSE'].round(6)
result_df['MSE'] = result_df['MSE'].round(6)
result_df['R2'] = result_df['R2'].round(6)
result_df['MAE'] = result_df['MAE'].round(6)
result_df

result_df.to_csv('rice_yield_final_results.csv', index=False)

In [ ]:
#RF-GBDT-SVR Ensemble Regressor
base_models = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42, max_depth=20)),
    ('gbdt', GradientBoostingRegressor(n_estimators=300, learning_rate=0.01, subsample= 0.9,max_depth=5, min_samples_leaf= 4, random_state=42)),
    ('svr', SVR(kernel='rbf', C=10, gamma='scale', epsilon=.1, degree=2))
]

stacking_regressor = StackingRegressor(estimators=base_models)

stacking_regressor.fit(X_train, y_train)

y_pred = stacking_regressor.predict(X_test)
r2 = r2_score(y_test, y_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute score: {mae}')
print(f'R2 score: {r2}')
print(f'RMSE score: {test_rmse}')
print(f'MSE score: {mse}')

X_cv = scaler.fit_transform(X)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(stacking_regressor, X_cv, y, cv=kf)
print(f"Cross-validated scores: {cv_scores}")
print(f"Mean CV score: {np.mean(cv_scores)}")

In [ ]:
#Feature Importance Evaluvation
importance = rf_model.feature_importances_

feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importance
})

feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

feature_importance

In [ ]:
#Best Parameters for Random Forest
rf = RandomForestRegressor(random_state=42)

param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [10, 15, 20]
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(f"Best Parameters: {best_params}")

best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error with the best parameters: {mse}")
print(f"R2 score with the best parameters: {r2}")

In [ ]:
#Best Parameters for SVR
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVR())
])

param_grid = {
    'svr__kernel': ['linear', 'rbf', 'poly'],
    'svr__C': [0.1, 1, 10, 100],
    'svr__gamma': ['scale', 'auto'],
    'svr__degree': [2, 3, 4]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error with the best parameters: {mse}")
print(f"R2 score with the best parameters: {r2}")

In [ ]:
#Best Parameters for XGB
param_grid = {
    'n_estimators': [100, 200, 300,400,500],
    'max_depth': [3, 4, 5,6,7,8],
    'learning_rate': [0.1,0.15,0.2],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}


model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)
r2 = r2_score(y_test, y_pred)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error with the best parameters: {mse}")
print(f"R2 score with the best parameters: {r2}")

In [ ]:
#Best Parameters for GBDT
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 0.9],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

model = GradientBoostingRegressor(random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(X_train, y_train)


best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error with the best parameters: {mse}")
print(f"R2 score with the best parameters: {r2}")

In [ ]:
#Correlation Matrix
corr_matrix = data[['Rainfall', 'Evapotranspiration', 'Soil_Moisture', 'Maxtemp','Mintemp','Avgtemp','Humid','Yield']].corr()

new_column_names = {
    'Rainfall': 'Rainfall',
    'Evapotranspiration': 'Evapotranspiration',
    'Soil_Moisture': 'Soil Moisture',
    'Maxtemp': 'Maximum Temperature',
    'Mintemp': 'Minimum Temperature',
    'Avgtemp': 'Average Temperature',
    'Humid': 'Humidity',
    'Yield': 'Rice Yield'
    
}

corr_matrix.rename(columns=new_column_names, index=new_column_names, inplace=True)

plt.figure(figsize=(9, 7))

sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, square=True, linewidths=0.5)

plt.title('Heatmap of Dependant variables and Rice Yield Correlation')
plt.tight_layout()
#plt.savefig('Rice_Yield_Correlation.png')
plt.show()